In [1]:
import csv
from time import time
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
class MODE():
    
    # Import data and partition it
    def __init__(self, directory):
        self.data = sc.textFile(directory).map(lambda x: (int(x.split(" "*3)[1]),x.split(" "*3)[0])).partitionBy(101) \
                      .persist()
        
    # Use combiner and reduce to find the mode    
    def income_mode(self):
        s = self.data.map(lambda x: (x[1],x[0])) \
                     .combineByKey((lambda x: (x,1)),
                                   (lambda x,y: (x[0], x[1]+1)),
                                   (lambda x,y: x if x[1] > y[1] else (y if x[1] < y[1] else min(x,y))))
        return s
    
    # Save results as CSV 
    def save_file(self, file_name):
        with open(file_name, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(self.income_mode().collect())   

In [3]:
data = MODE(r"C:\Users\taylankabbani2019\Downloads\Income\*")

print(data.income_mode().take(5))

# data.save_file('City_income_mode.txt')

[('city72', (100, 18523)), ('city38', (40, 590)), ('city68', (47, 985)), ('city45', (48, 2509)), ('city95', (46, 2433))]
